In [130]:
import pandas as pd
import mysql.connector
import datetime

In [131]:
df = pd.read_csv("sql_cw.csv")

In [132]:
df.columns = df.columns.str.strip()

In [133]:
def convert_time(s):
    try:
        s = str(s).strip().replace(".", ":").replace("a.m", "AM").replace("p.m", "PM")
        return datetime.datetime.strptime(s, "%I:%M %p").time()
    except:
        return None

In [134]:
def clean_mirror_size(value):
    if pd.isna(value) or str(value).strip() in ["-", " -", "- ", " - "]:
        return "Standard"
    return str(value).strip()

In [135]:
def generate_custom_id(existing_ids, prefix):
    max_num = 0
    for eid in existing_ids:
        if str(eid).startswith(prefix):
            try:
                num = int(eid.replace(prefix, ""))
                max_num = max(max_num, num)
            except:
                continue
    return f"{prefix}{str(max_num + 1).zfill(3)}"

In [136]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2004",
    database="salon_db"
)
cursor = conn.cursor()

In [137]:
def fetch_ids(table, column):
    cursor.execute(f"SELECT {column} FROM {table}")
    return [row[0] for row in cursor.fetchall()]

In [138]:
existing_ids = {
    "salon": fetch_ids("Salon", "salon_id"),
    "rush_day": fetch_ids("RushDays", "rush_day_id"),
    "mirror": fetch_ids("Mirror", "mirror_id"),
    "seat": fetch_ids("Seats", "seat_id"),
    "washroom": fetch_ids("Washroom", "washroom_id"),
    "pricing": fetch_ids("Pricing", "pricing_id"),
    "offer": fetch_ids("Offers", "offer_id"),
    "appointment": fetch_ids("Appointments", "appointment_id"),
    "method": fetch_ids("AppointmentMethods", "method_id"),
    "complimentary": fetch_ids("ComplimentaryItems", "item_id"),
    "facility": fetch_ids("Facilities", "facility_id"),
    "equipment": fetch_ids("SalonEquipments", "equipment_id"),
    "waiting": fetch_ids("WaitingSeats", "waiting_id"),
    "parking_area": fetch_ids("ParkingArea", "parking_area_id"),
    "parking_type": fetch_ids("ParkingType", "parking_type_id"),
    "tv": fetch_ids("TVSetup", "tv_id"),
    "access": fetch_ids("Accessibility", "accessibility_id"),
    "payment": fetch_ids("PaymentMethods", "payment_id"),
    "service": fetch_ids("CommonServices", "service_id")
}


insert data to the table


In [139]:
df.columns = df.columns.str.strip()

In [140]:
for i, row in df.iterrows():
    try:
        salon_id = generate_custom_id(existing_ids["salon"], "S")
        existing_ids["salon"].append(salon_id)

        open_start, open_end, rush_start, rush_end = None, None, None, None
        if "to" in str(row["Open_hours"]):
            parts = row["Open_hours"].split("to")
            if len(parts) == 2:
                open_start = convert_time(parts[0])
                open_end = convert_time(parts[1])
        if "to" in str(row["Rush_hours"]):
            parts = row["Rush_hours"].split("to")
            if len(parts) == 2:
                rush_start = convert_time(parts[0])
                rush_end = convert_time(parts[1])

        cursor.execute("""
            INSERT INTO Salon (salon_id, name, address, location, contact_number, open_days, open_hour_start, open_hour_end, rush_hour_start, rush_hour_end)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            salon_id, row["Name"], row["Address"], row["Location"], str(row["Contact_number"]),
            row["Open_days"], open_start, open_end, rush_start, rush_end
        ))
        conn.commit()

        if pd.notna(row["Rush_days"]):
            for day in str(row["Rush_days"]).split(","):
                day = day.strip().capitalize()
                rush_day_id = generate_custom_id(existing_ids["rush_day"], "RD")
                existing_ids["rush_day"].append(rush_day_id)
                cursor.execute(
                    "INSERT INTO RushDays (rush_day_id, salon_id, day_name) VALUES (%s, %s, %s)",
                    (rush_day_id, salon_id, day)
                )
                conn.commit()

        mirror_id = generate_custom_id(existing_ids["mirror"], "M")
        existing_ids["mirror"].append(mirror_id)
        mirror_size = clean_mirror_size(row["Mirror_size"])
        cursor.execute("""
            INSERT INTO Mirror (mirror_id, salon_id, mirror_type, mirror_shape, mirror_size)
            VALUES (%s, %s, %s, %s, %s)
        """, (mirror_id, salon_id, row["Mirror_type"], row["Mirror_shape"], mirror_size))
        conn.commit()
        

        seat_id = generate_custom_id(existing_ids["seat"], "SE")
        existing_ids["seat"].append(seat_id)
        cursor.execute("""
            INSERT INTO Seats (seat_id, salon_id, seat_count, seat_brand, seat_color, seat_material,
                               has_footrest, has_headrest, adjustable)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            seat_id, salon_id, row["Seat_count"], row["Seat_brand"], row["Seat_color"], row["Seat_material"],
            row["Footrest"] == "yes", row["Headrest"] == "yes", row["Adjustable"] == "yes"
        ))
        conn.commit()

        washroom_id = generate_custom_id(existing_ids["washroom"], "W")
        existing_ids["washroom"].append(washroom_id)
        cursor.execute("""
            INSERT INTO Washroom (washroom_id, salon_id, gender_type, trash_bins, air_freshener, hand_dryer)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (
            washroom_id, salon_id, row["Gender_Type"],
            row["Trash_Bins"] == "yes", row["Air_Freshener"] == "yes", row["Hand_Dryer"] == "yes"
        ))
        conn.commit()

        pricing_id = generate_custom_id(existing_ids["pricing"], "PR")
        existing_ids["pricing"].append(pricing_id)
        cursor.execute("""
            INSERT INTO Pricing (pricing_id, salon_id, min_price, max_price, price_range)
            VALUES (%s, %s, %s, %s, %s)
        """, (
            pricing_id, salon_id, row["Min_price"], row["Max_price"], row["Avarage_bill"]
        ))
        conn.commit()

        offer_id = generate_custom_id(existing_ids["offer"], "O")
        existing_ids["offer"].append(offer_id)
        cursor.execute("""
            INSERT INTO Offers (offer_id, salon_id, discount_policy, ongoing_offers)
            VALUES (%s, %s, %s, %s)
        """, (
            offer_id, salon_id, row["Discount_policy"] == "yes", row["Ongoing_offers"] == "yes"
        ))
        conn.commit()

        appointment_id = generate_custom_id(existing_ids["appointment"], "A")
        existing_ids["appointment"].append(appointment_id)
        cursor.execute("""
            INSERT INTO Appointments (appointment_id, salon_id, appointment_reminders, service_access_type)
            VALUES (%s, %s, %s, %s)
        """, (
            appointment_id, salon_id, row["Appointment_reminders"] == "yes", row["Service_access_type"]
        ))
        conn.commit()

        if pd.notna(row["appointment_method"]):
            for method in str(row["appointment_method"]).split(","):
                method_id = generate_custom_id(existing_ids["method"], "AM")
                existing_ids["method"].append(method_id)
                cursor.execute("INSERT INTO AppointmentMethods (method_id, salon_id, method_name) VALUES (%s, %s, %s)",
                               (method_id, salon_id, method.strip()))
                conn.commit()

        facility_id = generate_custom_id(existing_ids["facility"], "F")
        existing_ids["facility"].append(facility_id)
        cursor.execute("""
            INSERT INTO Facilities (facility_id, salon_id, wifi_available, air_conditioned, tv_available, charging_ports)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (
            facility_id, salon_id,
            row["WiFi_Availability"] == "yes",
            row["Air_condition"] == "yes",
            pd.notna(row["TV_Brand"]),
            row["Charging_ports.1"] == "yes"
        ))
        conn.commit()

        equipment_columns = [
        "Hair_steamer", "Hair_Dryer", "Neck_Duster", "Razor", "Towels", "Scissors",
        "Comb", "Curling_iron", "Flat_iron", "Clippers", "Powder_brush",
        "Beard_Liner_Tool", "Tool_rack"
                        ]

        for col in equipment_columns:
            if col in row and pd.notna(row[col]):
                try:
                    count = int(row[col])
                    if count > 0:
                        equipment_id = generate_custom_id(existing_ids["equipment"], "EQ")
                        existing_ids["equipment"].append(equipment_id)
                        cursor.execute("""
                            INSERT INTO SalonEquipments (equipment_id, salon_id, equipment_name, equipment_count)
                            VALUES (%s, %s, %s, %s)
                        """, (equipment_id, salon_id, col.replace("_", " "), count))
                        conn.commit()
                except ValueError:
                    print(f" Skipped invalid count in {col} for salon {salon_id}")
        
        
        waiting_id = generate_custom_id(existing_ids["waiting"], "WSEAT")
        existing_ids["waiting"].append(waiting_id)
        cursor.execute("""
            INSERT INTO WaitingSeats (waiting_id, salon_id, seat_count, seat_type)
            VALUES (%s, %s, %s, %s)
         """, (
            waiting_id, salon_id, row["Waiting_seat_Count"], row["Waiting_seat_type"]
         ))
        conn.commit()


        parking_type_name = row["Parking_type"]
        is_covered = row["Parking_covered"] == "yes"
        parking_type_id = None

        cursor.execute("SELECT parking_type_id FROM ParkingType WHERE type_name = %s", (parking_type_name,))
        result = cursor.fetchone()
        if result:
            parking_type_id = result[0]
        else:
            parking_type_id = generate_custom_id(existing_ids["parking_type"], "PT")
            existing_ids["parking_type"].append(parking_type_id)
            cursor.execute("INSERT INTO ParkingType (parking_type_id, type_name, is_covered) VALUES (%s, %s, %s)",
                           (parking_type_id, parking_type_name, is_covered))
            conn.commit()

        parking_area_id = generate_custom_id(existing_ids["parking_area"], "PA")
        existing_ids["parking_area"].append(parking_area_id)
        cursor.execute("""
            INSERT INTO ParkingArea (parking_area_id, salon_id, parking_capacity, parking_type_id)
            VALUES (%s, %s, %s, %s)
        """, (
            parking_area_id, salon_id, row["Parking_Capacity"], parking_type_id
        ))
        conn.commit()


        if pd.notna(row["TV_Brand"]) and pd.notna(row["TV_count"]):
            tv_id = generate_custom_id(existing_ids["tv"], "TV")
            existing_ids["tv"].append(tv_id)
            cursor.execute("""
                INSERT INTO TVSetup (tv_id, salon_id, brand, count)
                VALUES (%s, %s, %s, %s)
            """, (
                tv_id, salon_id, row["TV_Brand"], int(row["TV_count"])
            ))
            conn.commit()


        access_id = generate_custom_id(existing_ids["access"], "ACC")
        existing_ids["access"].append(access_id)
        cursor.execute(
        "INSERT INTO Accessibility (accessibility_id, salon_id, wheelchair_access) VALUES (%s, %s, %s)",
        (access_id, salon_id, str(row.get("Wheelchair_access", "")).strip().lower() == "yes")
        )
        conn.commit()



        if pd.notna(row.get("Payment_methods")) and str(row["Payment_methods"]).strip().lower() != "no":
            for method in str(row["Payment_methods"]).split(","):
                payment_id = generate_custom_id(existing_ids["payment"], "PM")
                existing_ids["payment"].append(payment_id)
                cursor.execute(
                "INSERT INTO PaymentMethods (payment_id, salon_id, method) VALUES (%s, %s, %s)",
            (payment_id, salon_id, method.strip())
        )
        conn.commit()


        if pd.notna(row["Common_Services"]):
            for service in str(row["Common_Services"]).split(","):
                service_id = generate_custom_id(existing_ids["service"], "CS")
                existing_ids["service"].append(service_id)
                cursor.execute("INSERT INTO CommonServices (service_id, salon_id, service_name) VALUES (%s, %s, %s)",
                               (service_id, salon_id, service.strip()))
                conn.commit()

        if pd.notna(row.get("complimentary_items")) and str(row["complimentary_items"]).strip().lower() != "no":
            for item in str(row["complimentary_items"]).split(","):
                item_id = generate_custom_id(existing_ids["complimentary"], "CI")
                existing_ids["complimentary"].append(item_id)
                cursor.execute(
                   "INSERT INTO ComplimentaryItems (item_id, salon_id, item_name) VALUES (%s, %s, %s)",
            (item_id, salon_id, item.strip())
        )
        conn.commit()


    except Exception as e:
        conn.rollback()
        print(f" Error on row {i} ({row.get('Name', 'unknown')}):", e)

conn.commit()
cursor.close()
conn.close()
print(" All data inserted and connection closed.")


 Error on row 3 (cut n shave): 1054 (42S22): Unknown column 'nan' in 'field list'
 Error on row 5 (La Passion salon): 1054 (42S22): Unknown column 'nan' in 'field list'
 All data inserted and connection closed.
